In [17]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets
from sklearn.metrics import classification_report
from tqdm import tqdm
from time import time
from sklearn.metrics import accuracy_score

In [18]:
#change to binary value- 0 for empty, 1 for #/+
def binary_conv(source):
    data=[]
    for s in source:
        text=np.array(list("".join(s)))
        data.append(np.where(text==' ' ,0 ,1))
    return data

In [19]:
#load labels data
training_labels=np.genfromtxt('traininglabels')
validation_labels=np.genfromtxt('validationlabels')
test_labels=np.genfromtxt('testlabels')

In [20]:
#load images data
training_images_shape=np.loadtxt('trainingimages', dtype='str', delimiter='\n', comments=None).reshape(len(training_labels),-1)
validation_images_shape=np.loadtxt('validationimages', dtype='str', delimiter='\n', comments=None).reshape(len(validation_labels),-1)
test_images_shape=np.loadtxt('testimages', dtype='str', delimiter='\n', comments=None).reshape(len(test_labels),-1)

In [21]:
training_images=binary_conv(training_images_shape)
validation_images=binary_conv(validation_images_shape)
test_images=binary_conv(test_images_shape)

# Q1 - Naïve Bayes

#### Helper Function:

In [41]:
def getCondProbs(trainingDigits,traindata,trainlabel,k):
    condProb = {} # dict of - {digit: conditional prob}
    for digit in trainingDigits:
        for trainlist,label in zip(traindata,trainlabel):
            if(label==digit):
                if(label not in condProb.keys()):
                        count_pixel_list = [0] * 784 # build new list 
                        condProb[digit] = count_pixel_list
                for index in range(len(trainlist)):
                    condProb[digit][index] += trainlist[index] # add the value (0/1)
    for key in condProb.keys(): # go over all digits
        temp_list = condProb.get(key) 
        condProb[key] = [(x + k)/(trainlabel.count(key)+2*k) for x in temp_list] # add smoothing (2= Number of classes {0,1})
    return condProb
                                                   
def computeLikelihood(testDigit, condProbCounters): # function ger test simple digit and compute prob
    likelihood_list = []
    for pixel in range(len(testDigit)):
        likelihood_list.append(testDigit[pixel] * condProbCounters[pixel]) # the val on this cell * condprob
    return likelihood_list

In [42]:
def naive_bayes(traindata,trainlabel,testdata,testlabel,k): # k = smoothing
    digits = [i for i in range(10)] # digits - {0,9}
    c_prob=[]                        #list to store P(class)
    y_pred = []                      # list for the prediction
    d_acc=[]                         #Take a list that will save each class(digit) accuracy
    cond_prob = getCondProbs(digits,traindata,trainlabel,k)  #get dict with digit and her conditional prob
    
    for c in digits:
        c_prob.append(trainlabel.count(c)/len(trainlabel)) # the prob of each digit from training data = P(C)
    
    for i in range(len(testdata)): # go through all the test samples
        lists=[]   #Empty list to store probability of all digit for every test simple
        for j in range(10): # {0-9}  
            cond_prob_of_digit = computeLikelihood(testdata[i],cond_prob.get(j))  # compute likelihood for each digit
            cond_prob_of_digit = list(filter(lambda a: a != 0, cond_prob_of_digit)) #remove 0 from prob list
            score=np.sum(np.log(cond_prob_of_digit)+np.log(c_prob[j])) 
            lists.append(score) #Append Probability of jth class for ith simple
        pred=lists.index(max(lists)) # get the index (y) for max score = predict
        y_pred.append(pred)
      
    accuracy = accuracy_score(testlabel, y_pred)  # compute accuracy      
    return accuracy

## A. Validation for value K : [ 1-5 ] 

In [43]:
#Call Naive Bayes Function
accuracy=naive_bayes(training_images,training_labels.tolist(),validation_images,validation_labels.tolist(),1)

#Print Overall Accuracy
print('Accuracy For K=1: '+str(accuracy))

Accuracy For K=1: 0.706


In [44]:
#Call Naive Bayes Function
accuracy=naive_bayes(training_images,training_labels.tolist(),validation_images,validation_labels.tolist(),2)

#Print Overall Accuracy
print('Accuracy For K=2: '+str(accuracy))

Accuracy For K=2: 0.705


In [45]:
#Call Naive Bayes Function
accuracy=naive_bayes(training_images,training_labels.tolist(),validation_images,validation_labels.tolist(),3)

#Print Overall Accuracy
print('Accuracy For K=3: '+str(accuracy))

Accuracy For K=3: 0.708


In [46]:
#Call Naive Bayes Function
accuracy=naive_bayes(training_images,training_labels.tolist(),validation_images,validation_labels.tolist(),4)

#Print Overall Accuracy
print('Accuracy For K=4: '+str(accuracy))

Accuracy For K=4: 0.704


In [47]:
#Call Naive Bayes Function
accuracy=naive_bayes(training_images,training_labels.tolist(),validation_images,validation_labels.tolist(),5)

#Print Overall Accuracy
print('Accuracy For K=5: '+str(accuracy))

Accuracy For K=5: 0.702


In [48]:
def df_style(val):
    if(val == "0.708" or val == "3"):
        return "font-weight: bold"

data = [["1","0.706"],["2","0.705"],["3","0.708"],["4","0.704"],["5","0.702"]]
df = pd.DataFrame(data, columns=["K-Value","Accuracy"])

df.style.hide_index().applymap(df_style)

K-Value,Accuracy
1,0.706
2,0.705
3,0.708
4,0.704
5,0.702


## B. Predict with K=3:

In [49]:
#Call Naive Bayes Function
accuracy=naive_bayes(training_images,training_labels.tolist(),test_images,test_labels.tolist(),3)

#Print Overall Accuracy
print('Accuracy For K=3 with the test data: '+str(accuracy))

Accuracy For K=3 with the test data: 0.699


## C. Predict with K=3:

In [50]:
accuracy=naive_bayes(training_images,training_labels.tolist(),training_images,training_labels.tolist(),3)

print('Accuracy For K=3 with the training data: '+str(accuracy))

Accuracy For K=3 with the training data: 0.7376


# Q2 - Perceptron

In [30]:
# add bias to the first index on lists
def add_bias_to_vector(vectors,bias):
    for i in range(len(vectors)):
        if(isinstance(vectors[i],list)):
            vectors[i] = [bias] + vectors[i]
        else:
            vectors[i] = [bias] + vectors[i].tolist()
    return vectors

In [31]:
def train_weights(traindata,trainlabel,num_iterations,learning_rate=1):

    # initialize weights to 0
    weights = []
    for i in range(10):
        weights.append([0]*(28*28 +1))
    
    done = False #check if we got zero error
    count = 0 # number of iterations

    while(not done and count < num_iterations):
        changed = 0 # if zero then no error
        for i in range(len(traindata)):
            #val stores the values returned by each perceptron for numbers 0 - 9
            val = [0]*10
            
            max = float('-inf')
            
            # then loop through other perceptrons, if any of them have a higher val returned they are our prediction 
            for digit in range(10):
                score = np.dot(traindata[i] , weights[digit]) # compute score
                if score > max:
                    max = score
                    prediction = digit # save the digit
                # Takes the smallest digit in case of equality
                elif(score == max):
                    if(digit < prediction):
                        max = score
                        prediction = digit
                        
            #if the prediction we made was wrong we need to update the weights. otherwise, just continue 
            if prediction != trainlabel[i]:
                changed = changed + 1 
                weights[int(trainlabel[i])] = np.add(weights[int(trainlabel[i])], traindata[i] * learning_rate) # (W_y + F)
                weights[prediction] = np.subtract(weights[prediction], traindata[i] * learning_rate) # (W_y' - F)
        count = count + 1
        if (changed == 0): # if no change, so we stop the interation
            done = True
    return weights

def perceptron(traindata,trainlabels,testdata,testlabels,num_iterations,learning_rate=1):
    
    update_weights = train_weights(traindata,trainlabels,num_iterations,learning_rate) # got the weights after training
    y_pred = []
    
    for i in range(len(testdata)): # go over test simple
        max = float('-inf')
        for digit in range(10):
            score = np.dot(testdata[i], update_weights[digit]) # compute score
            if(score > max): # take the max score
                digit_pred = digit
                max = score
            elif (score == max):
                if(digit < digit_pred):
                    digit_pred = digit
                    max = score    
        y_pred.append(digit_pred)
    accuracy = accuracy_score(testlabels, y_pred)   
    return accuracy

In [32]:
training_images=add_bias_to_vector(training_images,1)
test_images=add_bias_to_vector(test_images,1)

### A. For epoch = 3 : 

In [33]:
print('Accuracy for epochs = 3 with train data: ' +str(perceptron(training_images,training_labels,training_images,training_labels,3,1)))

Accuracy for epochs = 3 with train data: 0.852


In [46]:
print('Accuracy for epochs = 3 with test data: ' +str(perceptron(training_images,training_labels,test_images,test_labels,3,1)))

Accuracy for epochs = 3 with test data: 0.763


### B. For epoch 1-5 compare to <u>train</u> data: 

In [47]:
for i in range(1,6):
    print('Accuracy for epochs = ' + str(i) + ' : ' +str(perceptron(training_images,training_labels,training_images,training_labels,i,1)))
    print()

Accuracy for epochs = 1 : 0.87

Accuracy for epochs = 2 : 0.891

Accuracy for epochs = 3 : 0.852

Accuracy for epochs = 4 : 0.9112

Accuracy for epochs = 5 : 0.8406



### For epoch 1-5 compare to <u>test</u> data: 

In [48]:
for i in range(1,6):
    print('Accuracy for epochs = ' + str(i) + ' : ' +str(perceptron(training_images,training_labels,test_images,test_labels,i,1)))
    print()

Accuracy for epochs = 1 : 0.781

Accuracy for epochs = 2 : 0.812

Accuracy for epochs = 3 : 0.763

Accuracy for epochs = 4 : 0.814

Accuracy for epochs = 5 : 0.746

